In [ ]:
#| default_exp bridge

In [ ]:
#| export
from __future__ import annotations

# Bridge helpers

Bridget is meant to be used interactively. In most cases running all cells should (and hopefully will) work Ok. But this notebook in particular is developing the bridge itself. It has lot of test/debug code and cells creating and re-creating widgets. `ipywidgets` and `AnyWidget` are finicky in this kind of situations. It also probably won't behave exactly in VSCode or Lab.

<!-- WARNING: this notebook has many blocking calls to the frontend. Commands running all/above/below cells can be slow or fail (see [BlockingMixin](./10_bridge_widget.ipynb)). Simply run again the offending cell. -->

<!-- # Prologue -->

In [ ]:
#| hide
import ipywidgets as W

In [ ]:
#| hide
W.Widget.close_all()  # we don't want stale widgets around when developing

In [ ]:
#| export
from collections.abc import Sequence
from pathlib import Path
from typing import Any
from typing import overload
from urllib.parse import ParseResult

import anywidget
import fastcore.all as FC
import traitlets as T
from fastcore.all import L
from fastcore.all import NotStr
from fastcore.xml import escape
from fastcore.xml import to_xml
from fasthtml.core import FT
from fasthtml.xtend import Script
from IPython.display import clear_output
from IPython.display import display
from pote.basic import bundle_path
from pote.common import shorten
from pote.common import shortens
from pote.common import update_


In [ ]:
#| hide
import os

In [ ]:
#| hide
os.environ['DEBUG_BRIDGET'] = 'True'

In [ ]:
#| export

from fasthtml.components import B, Details, Summary, Pre

In [ ]:
#| export
from bridget.bridge_widget import anysource
from bridget.bridge_widget import blocks
from bridget.bridge_widget import BridgeWidget
from bridget.bridge_widget import bundled
from bridget.bridge_widget import resolve_ESM
from bridget.helpers import Brd_Mark
from bridget.helpers import brdmark_js
from bridget.helpers import bridge_cfg
from bridget.helpers import DEBUG
from bridget.helpers import HTML
from bridget.helpers import id_gen
from bridget.logger import FCanvas
from bridget.logger import FCanvas_css
from bridget.logger import FLogger
from bridget.logger import NBLogger
from bridget.logger import NoopLogger
from bridget.nb import NB


In [ ]:
#| hide

import time

import nbdev.config
from fastcore.test import *
from pote.common import setup_console
from pote.widget import cleanupwidgets


In [ ]:
#| hide

from bridget.bridge_widget import _show
from bridget.bridge_widget import ablocks


In [ ]:
#| hide

from fasthtml.components import Div, P, Pre, Text, Span

----

In [ ]:
#| hide

IDISPLAY = display
_n = '\n'
console, cprint = setup_console(140)
# test_is(DEBUG(), True)

In [ ]:
#| exporti

new_id = id_gen()
BUNDLE_PATH = bundle_path(__name__)
# bridge_cfg.for_module(__name__)

In [ ]:
#| hide
%env ANYWIDGET_HMR=0

env: ANYWIDGET_HMR=0


----

# Helpers
> Internal utilities for JS/Python interop. Most users won't need these directly.

In [ ]:
#| export

def debug(jsstr: str): return f"debugger;\n{jsstr}"
def notdebug(jsstr: str): return jsstr

In [ ]:
#| export

def _to_js(ft):
    if isinstance(ft, FT): 
        return [ft.tag, [_to_js(_) for _ in ft.children], ft.attrs]
    return str(ft)

def to_js(*fts: FT|str):
    return f"[\n  {',\n  '.join(repr(_to_js(_)) if isinstance(_, FT) else repr(_) for _ in fts)}\n]"

In [ ]:
a = Div(P('a'), P('aa'), id='a11')
b = P('b', Span('bb'))

test_eq(_to_js(a), ['div', [['p', ['a'], {}], ['p', ['aa'], {}]], {'id': 'a11'}])
test_eq(_to_js(b), ['p', ['b', ['span', ['bb'], {}]], {}])

test_eq(to_js(a, 'const a=`${"a"}`; console.log(a)', b), """[
  ['div', [['p', ['a'], {}], ['p', ['aa'], {}]], {'id': 'a11'}],
  'const a=`${"a"}`; console.log(a)',
  ['p', ['b', ['span', ['bb'], {}]], {}]
]""")

In [ ]:
a = Script('const a=`${"a"}`; console.log(a)')
b = Script(src='https://unpkg.com/htmx.org@next/dist/htmx.js', type='module')

test_eq(repr(_to_js(a)), "['script', ['const a=`${\"a\"}`; console.log(a)'], {}]")

test_eq(to_js(a, b), """[
  ['script', ['const a=`${"a"}`; console.log(a)'], {}],
  ['script', [''], {'src': 'https://unpkg.com/htmx.org@next/dist/htmx.js', 'type': 'module'}]
]""")

Quick & dirty way to convert `FT` to HTML elements int the front-end using `fasthtml-js` `$E`.  
Intended only for "linking"/head elements (with void or text content): script, style, link, meta, etc. It'll surely fail with other elements.

In [ ]:
#| exporti

def _safe(o, m='r', l=140): return escape(shorten(o, m, l), False)  # type: ignore

In [ ]:
#| export

class ScriptsDetails:
    def __init__(self, scs, title='Loaded scripts', open=True): 
        self.scs = scs; self.title = title; self.open = open
    def __ft__(self):
        return Details(open=self.open)(
            Summary(B(self.title)),
            Pre(NotStr('\n'.join(escape(to_xml(_, indent=False, do_escape=False).strip()) for _ in self.scs))),
        )

# Bridge scripts

`HTMX` and other useful JS libraries.

In [ ]:
#| export

def _bridge_scripts_extra():
    from fasthtml.core import surrsrc, scopesrc
    return {'surreal': surrsrc, 'css_scope_inline': scopesrc}

def bridge_scripts(htmx=True):
    from fasthtml.core import fhjsscr
    from fasthtml.xtend import Script
    htmxsrc = Script(src=f"https://unpkg.com/htmx.org@next/dist/htmx.{'' if DEBUG() else 'min.'}js")()
    return update_({'htmx': htmxsrc } if htmx else {}, fasthtml_js=fhjsscr, **_bridge_scripts_extra())

def show_scripts(**scs: FT): 
    display(HTML(ScriptsDetails(shortens(map(to_xml, scs.values()), 'r', 140))))

In [ ]:
bridge_scripts()

{'htmx': script(('',),{'src': 'https://unpkg.com/htmx.org@next/dist/htmx.js'}),
 'fasthtml_js': script((),{'src': 'https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js@1.0.12/fasthtml.js'}),
 'surreal': script((),{'src': 'https://cdn.jsdelivr.net/gh/answerdotai/surreal@main/surreal.js'}),
 'css_scope_inline': script((),{'src': 'https://cdn.jsdelivr.net/gh/gnat/css-scope-inline@main/script.js'})}

In [ ]:
show_scripts(**bridge_scripts())

# BCanvas
> `FCanvas` associated with a bridge.

Configure the bridge logging system to display log traces here in the notebook, a convenience that alleviates the need to open the browser console.

NOTE: BCanvas could be a `Bridge` plugin, but it's a separate widget instead in order to use another comm channels. (must check if we gain something by having different Comm channels, in the end ZMQD [uses](https://jupyter-server.readthedocs.io/en/latest/developers/websocket-protocols.html) one websocket under the hood, I think)

In [ ]:
#| hide
bcanvas_js = BUNDLE_PATH / 'js/bcanvas.js'
bundled(bcanvas_js)();

bridge_js = BUNDLE_PATH / 'js/bridge.js'
bundled(bridge_js)();

In [ ]:
#| exporti

class BCanvas(FCanvas):
    _esm = '''
const { bcanvas } = await brdimport('./bcanvas.js');
export default { initialize({ model }) {
  bcanvas.setup(model);
  model.set('_loaded', true); model.save_changes();
  return () => {
    bcanvas.setup();
    model.set('_loaded', false); model.save_changes();
  }
}}
'''
    _css = FCanvas_css

In [ ]:
#| export

__bcanvas__ = None

@FC.delegates(BCanvas, keep=True)  # type: ignore
def get_bcanvas(**kwargs):
    global __bcanvas__
    if not __bcanvas__:
        timeout, sleep = kwargs.pop('timeout', 2), kwargs.pop('sleep', 0.2)
        __bcanvas__ = BCanvas.create(timeout=timeout, sleep=sleep, **kwargs)
    return __bcanvas__

In [ ]:
cleanupwidgets('cnv')
__bcanvas__ = None

cnv = get_bcanvas(timeout=DEBUG(2, 2))                
# test_eq(cnv.loaded(), True)  # in Lab something weird is happening here

In [ ]:
cnv.show()

In [ ]:
cnv.add('Why Universe, why?<br>')

# BLogger
> `FLogger` subclass used by Bridge.

In [ ]:
#| export

class BLogger(FLogger):
    canvas: BCanvas

    def _setup_canvas(self, height:int): self._canvas = get_bcanvas(height=height)
    def close_canvas(self): ...
    def msg(self, msg, clear:bool=False, ctx:str='logger', **kwargs):
        super().msg(msg, clear=clear, ctx=ctx, format=self.canvas is None or (not self.canvas.displayed()), **kwargs)

In [ ]:
lgr = BLogger('BLogger initialized')

In [ ]:
lgr.log('test')

In [ ]:
lgr.error('test')

In [ ]:
lgr.warn('test')

In [ ]:
lgr.log('bbbb')

In [ ]:
lgr.log('cccccc', True)

In [ ]:
lgr.show()

In [ ]:
lgr.log('ddddddddd')

Log directly.

# Bridge bootstrap

Progressive bridge construction: Boot → Messenger → Bridge (with plugins).
Each layer adds functionality. Use `get_bridge()` to get the full-featured bridge.

In [ ]:
#| export

defaultLogConfig = {
    'ns': 'brd',
    'color': 'light-dark(gray, lightgray)', # #fdc400 darkgoldenrod gold
    'fmt': 'htmlFmt',
    'tsDelta': True,
    'INFO': {},
    'ERROR': {
        'color': 'red',
    },
    'WARN': {
        'color': 'LightSalmon',
    },
}

## BridgeBoot
> Simply setup the bridge and connect logging.

In [ ]:
#| exporti

__brd__ = None

def _get_bridge(): return __brd__

def _set_bridge(value):
    global __brd__
    __brd__ = value


class BridgeBoot(BridgeWidget):
    _esm = '''
const { bcanvas } = await brdimport('./bcanvas.js');
const { initBridge } = await brdimport('./bridge.js');

export default { async initialize({ model, experimental }) {
// debugger;
  try {
    const cb = await initBridge(model, experimental.invoke);
    model.set('_loaded', true); model.save_changes();
    return () => { cb(); model.set('_loaded', false); model.save_changes(); };
  } catch (e) {
    console.error('Error initializing bridge', e);
  }
}}
'''
    _css = FCanvas_css

    ctx_name = T.Unicode('brd').tag(sync=True)
    logger = T.Instance(NBLogger, default_value=NoopLogger())
    logger_config = T.Dict(default_value=defaultLogConfig).tag(sync=True)

    def __init__(self, logger=None, show_logger=False, **kwargs):
        if logger is None: logger = BLogger(show=show_logger)
        elif show_logger: self.logger.show()
        self.on_msg(self._message_hdlr)
        super().__init__(logger=logger, **kwargs)
    @T.observe('_loaded')
    def _on_loaded(self, change):
        _set_bridge(self if change['new'] else None)
        self.log(f"<strong>The bridge is {('closed', 'open')[change['new']]}</strong>")
    def close(self):
        if self.comm is not None:
            self.send(dict(ctx=self.ctx_name, cmd='remove'))
            self._on_loaded(FC.AttrDict(new=False))
        super().close()

    def _message_hdlr(self, _, content, buffers):
        self.log(f"handle message: content=<code>{_safe(content)}</code>"
                f"{' buffers=><code>'+_safe(buffers)+'</code>' if buffers else ''}")

    def update_logger_config(self, **kwargs): self.logger_config = { **self.logger_config, **kwargs }

    def log(self, msg:Any, clear:bool=False): self.logger.msg(f"*{msg}", clear=clear, ctx=self.ctx_name, level='LOG')
    def error(self, msg:Any, clear:bool=False): self.logger.msg(f"*{msg}", clear=clear, ctx=self.ctx_name, level='ERROR')
    def warn(self, msg:Any, clear:bool=False): self.logger.msg(f"*{msg}", clear=clear, ctx=self.ctx_name, level='WARN')

# for n,l in _l2l.items(): setattr(BridgeBoot, n, partialmethod(BridgeBoot._log, level=l))

In [ ]:
cleanupwidgets('brd')

brd = BridgeBoot.create(show_logger=True, timeout=DEBUG(2, 2), sleep=0.2)
test_eq(brd.loaded(), True)

Note `create` blocks, it waits for the widget to be loaded (or the default timeout, see `BlockingMixin`).

In [ ]:
brd.error('Erred!')

In [ ]:
brd.update_logger_config(color='darkgoldenrod')  # type: ignore

In [ ]:
brd.log('my trea~~sssure')

`lgr` now that the bridge is opened reflects both frontend and kernel log messages and uses the JS bridge logging functionality.

In [ ]:
scr = Script('''
const { bridge } = await brdimport('./bridge.js');
bridge.logger.log('Hi from JS land');
''', type='module')
display(HTML(scr))
time.sleep(0.1)
clear_output()

Log from JS land.

In [ ]:
@FC.delegates(BridgeBoot, keep=True)  # type: ignore
def get_bridge(logger:NBLogger|None=None, show_logger:bool=False, **kwargs) -> BridgeBoot:
    if not __brd__:
        timeout, sleep = kwargs.pop('timeout', 2), kwargs.pop('sleep', 0.2)
        brd = BridgeBoot.create(logger=logger, show_logger=show_logger, timeout=timeout, sleep=sleep, **kwargs)
        return brd
    assert __brd__ is not None
    brd:BridgeBoot = __brd__
    if logger: brd.logger = logger
    if show_logger: brd.logger.show()
    return brd

In [ ]:
brd = get_bridge(show_logger=True)
test_is(get_bridge(), brd)

In [ ]:
brd.close()
test_is(__brd__ is None, True)

`close()` disconnect the widget and remove the widget model from the JS bridge, so it can no longer communicate with the Python side. Besides that, it has no effect on `bcanvas`, or the modules imported with `brdimport`. Create a new bridge to reconnect.

We need to use AnyWidget to setup the JS bridge, a class. The python side of the bridge should be created once per notebook, no sense to have several bridges around.

Given the nearly impossibility of creating class singletons in Python even with metaclasses, in previous refactorizations, I've used `SingletonConfigurable` a-la-`getipython`. It works very well but it uglify the code.

Anyway, given we're-all-adults-here, you should always use `get_bridge` to access the bridge instance.

In [ ]:
brd = get_bridge(show_logger=True)

# Messages from the beyond

## handle_message

In [ ]:
#| exporti

def handle_message(
        o: Any, 
        *args: Any, 
        ctx:str, kind:str, 
        prefix:str='on_', forward:bool=True, forward_name:str='_msg_fwrdr', 
        **kwargs: Any) -> bool:
    """if `o` has an attr named `ctx`, look for a handler with the form `on_{kind}` 
    passing the rest of `msg` and `args` to it. 
    If `forward`and `o` has an attr named with `forward_name`, call it with `o`, `msg` and `args`."""
    # print(f"handle_message: {o=} {args=} {ctx=}, {kind=} {kwargs=}")
    handled = False
    if o:
        if ctx in getattr(o, 'ctx_names', ()) and (fn := getattr(o, f"{prefix}{kind}", None)): fn(*args, **kwargs); handled = True
        if forward and (fn := getattr(o, forward_name, None)): fn(*args, ctx=ctx, kind=kind, **kwargs); handled = True
    return handled

In [ ]:
class _A:
    ctx_names={'A'}
    def __init__(self): self.forward = _B()
    def on_info(self, *args, **kwargs): 
        print('A', 'info', f"{args=}", f"{kwargs=}")
    def _msg_fwrdr(self, *args, ctx:str, kind:str, **kwargs):
        if ctx == 'B':
            handle_message(self.forward, *args, ctx=ctx, kind=kind, **kwargs)
        else:
            print(f"{ctx=} Not forwarded")

class _B:
    ctx_names={'B'}
    def on_info(self, *args, **kwargs):
        print('B', 'info', f"{args=}", f"{kwargs=}")

a = _A()
test_stdout(
    lambda: handle_message(a, 'hello', ctx='A', kind='info', info='initialized'), 
    "A info args=('hello',) kwargs={'info': 'initialized'}\nctx='A' Not forwarded")
test_stdout(
    lambda: handle_message(a, 'other', ctx='B', kind='info', info='forwarded'), 
    "B info args=('other',) kwargs={'info': 'forwarded'}")

## BridgeMessenger
> A bridge that can receive messages from JS land.

BridgeBoot just loads the JS bridge and setup logging. Here we set the basis for more powerful messaging to and fro JS.

In [ ]:
#| exporti

@FC.delegates(BridgeBoot, keep=True)  # type: ignore
class BridgeMessenger(BridgeBoot):
    ctx_names: set[str]

    def __init__(self, **kwargs):
        self._pending = {}
        super().__init__(**kwargs)
        # Warning: As AnyWidget calls `self.add_traits()`, 
        # `self.__class__` before `super()__init__` is not `self.__class__` now
        self.ctx_names = set(L(type(self).mro()).attrgot('ctx_name.default_value').filter())

    def on_info(self, *args, info:str, **kwargs):
        "Handle 'info' messages from the front-end."
        if info == 'loaded':
            self.log(f"{self.__class__.__name__} loaded.")
            self.log(f"bridge_cfg: {vars(bridge_cfg)}")
        elif info == 'model-set':
            self.log(f"{self.__class__.__name__} model set.")
        elif info == 'model-unset':
            self.log(f"{self.__class__.__name__} model unset.")

    def on_error(self, *args, error:str, **kwargs):
        "Handle 'error' messages from the front-end."
        self.log(f"{self.__class__.__name__} error: {error}")

    def debug_enabled(self, ctx:str|None=None, enabled=True, **kwargs):
        "Switch debug logs."
        self.send({'ctx': self.ctx_name, 'cmd': 'debug', 
                'args': {'debug_ctx': ctx or self.ctx_name, 'enabled': enabled, **kwargs}})
    
    def msg(self, tracker:Any=None, **kwargs):
        "Compose a message with tracking."
        msg_id = new_id()
        if tracker: self._pending[msg_id] = tracker
        return update_(kwargs, msg_id=msg_id)
    def _message_hdlr(self, _, content, buffers):
        "Front-end message hub for all ctx handlers."
        self.log(f"handle message: content=<code>{_safe(content)}</code>"
                f"{' buffers=<code>'+_safe(buffers)+'</code>' if buffers else ''}")
        if 'ctx' in content:
            tracker = self._pending.get(msg_id := content.get('msg_id'))
            try: handle_message(self, buffers, **content, tracker=tracker)
            finally: self._pending.pop(msg_id, None)
    
    def _msg_fwrdr(self, *args, ctx:str, kind:str, **kwargs):
        "Forward message to the appropriate ctx handler."
        if ctx not in self.ctx_names:
            self.warn(f"Unknown forward ctx '{ctx}' {kind=} kwargs=<code>{_safe(kwargs)}</code>")

In [ ]:
@FC.delegates(BridgeMessenger, keep=True)  # type: ignore
def get_bridge(logger:NBLogger|None=None, show_logger:bool=False, **kwargs):
    if not __brd__: 
        timeout, sleep = kwargs.pop('timeout', 3), kwargs.pop('sleep', 0.2)
        brd = BridgeMessenger.create(logger=logger, show_logger=show_logger, timeout=timeout, sleep=sleep, **kwargs)
        return brd
    assert __brd__ is not None
    brd = __brd__
    if logger: brd.logger = logger
    if not logger and show_logger: brd.logger.show()
    return brd

In [ ]:
cleanupwidgets('brd')

brd = get_bridge(show_logger=True, timeout=DEBUG(2, 2))

In [ ]:
brd.debug_enabled('loader', enabled=True)

In [ ]:
brd.send(brd.msg(ctx='brd', cmd='echo', args='test'))

In [ ]:
msg = dict(ctx='loader', cmd='load', args={'confetti': 'https://esm.sh/canvas-confetti@1.6'}, timeout=DEBUG(1))
brd.send(msg)

In [ ]:
observer_loader = anysource('''
const { getObserverManager } = await brdimport('./observer.js');
console.log(getObserverManager)
''')

brd.send(dict(ctx='loader', cmd='load', args={'get_observer': observer_loader}, reload=True), timeout=DEBUG(1))

({'ctx': 'loader',
  'kind': 'load',
  'success': ['get_observer'],
  'failed': [],
  'msg_id': 'msg-8'},
 [])

In [ ]:
src = '''
const res = await brdimport('https://a.com/b/c.js');
console.log(res)
'''

brd.send(dict(ctx='loader', cmd='load', args={'willfail': src}, reload=True), timeout=DEBUG(1))

({'ctx': 'loader',
  'kind': 'load',
  'success': [],
  'failed': [{'name': 'willfail',
    'error': 'Failed to fetch dynamically imported module: https://a.com/b/c.js'}],
  'msg_id': 'msg-9'},
 [])

`anysource` doesn't transform its arguments. Use `bundled` (i.e., `Bundle.__call__(..., transform=True)`, the default), instead.

You can disabled JS transform completely with `bridge.cfg.bundle_cfg.rewrite_imports`.

In [ ]:
src = bundled('''
import confetti from 'https://esm.sh/canvas-confetti@1.6';
console.log(confetti)
''')(transform=False)

brd.send(dict(ctx='loader', cmd='load', args={'load_confetti': src}), timeout=DEBUG(1))

({'ctx': 'loader',
  'kind': 'load',
  'success': ['load_confetti'],
  'failed': [],
  'msg_id': 'msg-10'},
 [])

Check the JS console network tab, we imported the `confetti` module previously.

In [ ]:
src = anysource('''
const { confetti } = await brdimport('https://esm.sh/canvas-confetti@1.6');
console.log(confetti)
''')

brd.send(dict(ctx='loader', cmd='load', args={'load_confetti': src}, reload=True), timeout=DEBUG(1))

({'ctx': 'loader',
  'kind': 'load',
  'success': ['load_confetti'],
  'failed': [],
  'msg_id': 'msg-11'},
 [])

`brdimport` doesn't re-import `confetti` too.

Note the `reload` argument of the `send` method. `bridge` caches the modules by name and URL, and we're reusing `load_confetti` name.

Use `cache=False` if you don't want `bridge` caching the module.

# Linking elements loader
> `script` (and other links) loader for notebooks.

`FastHTML` way of loading `head` elements is fine with standard web apps, `head` links are evaluated in order (unless they have `async` or `defer` attributes) if present in the HTML source.

In notebooks, we need to load those `head` elements dinamically in order (in `head`, `body` or anywhere of the front-end page). And we want to load `fasthtml.js` (and `htmx.js` because why not) as soon as possible, so we can use them in the same cell to define our JS extensions.

Assuming we've already loaded `fasthtml.js`, this is a possible solution:

In [ ]:
#| export

def Links(*fts: FT):
    return Script(notdebug("""
if (window.$Ls) requestAnimationFrame(() => $Ls(%s));
""" % to_js(*fts)), type='module')

In [ ]:
#| export

def load_links(*fts: FT, feedback: str=''):
    dh = display(HTML(to_xml(Links(*fts)) + feedback), display_id=True)
    # if not feedback and dh: dh.update(HTML(''))
    if not feedback: clear_output()

In [ ]:
brd.logger.show()

In [ ]:
scr = Script(notdebug('''
const { bridge } = await brdimport('./bridge.js');
bridge.logger.log('silly script');
'''), id='silly-script', type='module')

load_links(scr)

In [ ]:
load_links(scr)  # see console, script was not loaded twice

`load_links` can be used to load any link element in the front-end in order (if bridge is active). It will auto delete the script after links are loaded if `feedback` is None so the link won't be reflected in the `.ipynb` file and loaded automatically on page open.

Unfortunately, in some Jupyter environments like VSCode, this only works if the cell is visible in the screen and is run interactively, not with `all below` or `all above`. VSCode only renders outputs that are visible. For an alternative, see `Loader` below.


In [ ]:
B('a')

```html
<b>a</b>
```

# BridgePlugin

In [ ]:
#| export

class BridgePlugin(FC.GetAttr):
    _default = 'bridge'; _xtra = ['send', 'asend', 'msg', '_pending']
    ctx_name: str
    bridge:Bridge  # type: ignore
    def __init__(self, ctx:str='', src:str|Path='', bridge=None):
        self.is_initialized = None
        self.ctx_name, self.src = ctx or getattr(self, 'ctx_name', ''), src or getattr(self, 'src', '')
        self.ctx_names = {self.ctx_name} | set(L(type(self).mro()).attrgot('ctx_name').filter()) | getattr(self, 'ctx_names', set())
        if bridge: self.bridge = bridge
    def on_init(self, *args, info:str, **kwargs):
        if info == 'initialized':
            self.is_initialized = True
            self.log(f"Plugin '{self.ctx_name}' initialized")
        else:
            self.is_initialized = False
            self.log(f"Can't initialize plugin: <code>{info}</code>")
    def log(self, msg:Any, clear:bool=False): self.bridge.logger.msg(f"*{msg}", clear=clear, ctx=self.ctx_name, level='LOG')
    def error(self, msg:Any, clear:bool=False): self.bridge.logger.msg(f"*{msg}", clear=clear, ctx=self.ctx_name, level='ERROR')
    def warn(self, msg:Any, clear:bool=False): self.bridge.logger.msg(f"*{msg}", clear=clear, ctx=self.ctx_name, level='WARN')

In [ ]:
plg = BridgePlugin('test', 'a=10')
test_eq(plg.ctx_name, 'test')
test_eq(plg.ctx_names, {'test'})
test_eq(plg.src, 'a=10')

# Bridge
> BridgeWidget + plugins to extend Bridge functionality in Python and/or JavaScript.

`BridgeWidget` contains the core functionality, logging and JS loading. All other stuff in this project will be developed with plugins.

In [ ]:
#| exporti

@FC.delegates(BridgeMessenger, keep=True)  # type: ignore
class Bridge(BridgeMessenger):
    state: NB
    def __init__(self, *plugins:BridgePlugin, kwplugins:dict[str, str]|None=None, **kwargs):
        self.plugins = {  # stubs for default plugins
            'loader': BridgePlugin('loader', bridge=self), 
            'htmx': BridgePlugin('htmx', bridge=self), 
            'fasthtmljs': BridgePlugin('fasthtmljs', bridge=self)}
        self._init_plugins = (plugins, kwplugins)
        super().__init__(**kwargs)

    def _msg_fwrdr(self, *args, ctx:str, kind:str, **kwargs):
        handled = False
        if ctx in self.plugins:
            handle_message(self.plugins[ctx], *args, ctx=ctx, kind=kind, **kwargs); handled = True
        # elif ctx not in self.ctx_names:
        #     self.warn(f"Unknown forward ctx '{ctx}' {kind=} {kwargs=}")
        for k,p in self.plugins.items():
            if k != ctx and ctx in p.ctx_names:
                handle_message(p, *args, ctx=k, kind=kind, **kwargs); handled = True
        if not handled: self.warn(f"Unknown forward ctx '{ctx}' {kind=} {kwargs=}")

    @property
    def loader(self) -> Loader: return self.plugins['loader']  # type: ignore
    def __getattr__(self, name:str):
        if name in self.plugins: return self.plugins[name]
        raise AttributeError(f"Bridge has no plugin '{name}'")

    def add_plugins(self, *plugins:BridgePlugin, kwplugins:dict[str, str]|None=None, wait:int=0):
        if plugins or kwplugins:
            kwplugins = kwplugins or {}
            args, pp = {}, plugins + tuple(BridgePlugin(*t) for t in kwplugins.items())
            self.log(f"Adding plugins: {', '.join(p.ctx_name for p in pp)}")
            for p in pp:
                p.bridge = self
                self.plugins[p.ctx_name] = p
                if p.src: args[p.ctx_name] = anysource(p.src)
                else: handle_message(self, None, ctx=p.ctx_name, kind='init', info='initialized')
            if args:
                self.send(self.msg(ctx='loader', cmd='loadPlugins', args=args))
                if wait: blocks(lambda: all(self.plugins[k].is_initialized != None for k in args), wait)
    
    def on_info(self, *args, info:str, **kwargs):
        super().on_info(*args, info=info, **kwargs)
        if info == 'loaded':
            self.add_plugins(*self._init_plugins[0], kwplugins=self._init_plugins[1] or {})

    @anywidget.experimental.command  # type: ignore
    def get_prop(self, spec:dict, buffers):
        ctx, name, prop = spec['ctx'], spec['name'], None
        if ctx in self.plugins:
            prop = getattr(self.plugins[ctx], name, None)
        else:
            self.warn(f"get_prop: unknown ctx '{ctx}' for {name=}")
        return prop, buffers

In [ ]:
def get_bridge(*plugins:BridgePlugin, kwplugins:dict[str, str]|None=None, 
    logger:NBLogger|None=None, show_logger:bool=False, **kwargs):
    "Get the bridge, creating it if not found."
    if not __brd__: 
        timeout, sleep = kwargs.pop('timeout', 2), kwargs.pop('sleep', 0.2)
        brd = Bridge.create(*plugins, kwplugins=kwplugins, logger=logger, show_logger=show_logger, 
                            timeout=timeout, sleep=sleep, **kwargs)
        return brd
    assert __brd__ is not None
    brd = __brd__
    if logger: brd.logger = logger
    if not logger and show_logger: brd.logger.show()
    return brd

In [ ]:
cleanupwidgets('brd')

brd = get_bridge(show_logger=True, timeout=DEBUG(2, 2))
test_eq(brd.plugins.keys(), set(('loader', 'htmx', 'fasthtmljs')))

In [ ]:
brd.add_plugins(badp := BridgePlugin('badp', '''export default function badp(bridge) { a = 1/0 }'''))
blocks(lambda: badp.is_initialized != None, 1)
test_is(brd.badp.is_initialized, False)

# InspectPlugin

In [ ]:
class InspectPlugin(BridgePlugin):
    src = '''
function inspect(msg) {
    const {ctx, kind} = msg;
    if (kind === 'echo') {
        bridge.logger.log('echo', msg);
        setTimeout(() => {
            bridge.model.send({ ctx: ctx, kind: 'echo', msg: msg, msg_id: msg?.msg_id })
        }, 100);
        return;
    }
    bridge.model.send({ ctx: 'inspect', kind: 'inspect', msg: msg, msg_id: msg?.msg_id })
}
export default function initializeInspect(bridge) {
    bridge.on('inspect', inspect);
    return () => bridge.off('inspect');
}
'''
    ctx_name = 'inspect'

    def on_inspect(self, *args, msg:Any, tracker:Any, **kwargs):
        self.log(f"{self.__class__.__name__} inspect: {msg=} {tracker=}")

In [ ]:
brd.logger.show(clear=True)

In [ ]:
brd.add_plugins(insp := InspectPlugin())
blocks(lambda: insp.is_initialized is not None, 3)
test_is('inspect' in brd.plugins, True)

`add_plugin` does not blocks. If needed, use `blocks` or `blocking` to ensure the plugin is loaded.

In [ ]:
# content, _ = await insp.asend(msg := insp.msg(ctx='inspect', kind='echo', data={'a': 1}))  # asend not working in Lab
content, _ = insp.send(msg := insp.msg(ctx='inspect', kind='echo', data={'a': 1}), timeout=DEBUG(1))
test_eq(content, {'ctx': 'inspect', 'kind': 'echo', 'msg': msg, 'msg_id': msg['msg_id']})

In [ ]:
content, _ = insp.send(msg := insp.msg({'tracker': 'test'}, ctx='inspect', kind='echo', data={'b': 2}), timeout=DEBUG(1))
test_eq(content, {'ctx': 'inspect', 'kind': 'echo', 'msg': msg, 'msg_id': msg['msg_id']})

In [ ]:
insp.src = '''
const logger = bridge.logger.create({ ns: 'inspect', color: 'green' });
function inspect(msg) {
    const {ctx, kind} = msg;
    if (kind === 'echo') {
        logger.log('echo', msg);
        setTimeout(() => {
            bridge.model.send({ ctx: ctx, kind: 'echo', msg: msg, msg_id: msg?.msg_id })
        }, 100);
        return;
    }
    bridge.model.send({ ctx: 'inspect', kind: 'inspect', msg: msg, msg_id: msg?.msg_id })
}
export default async function initializeInspect(bridge) {
    bridge.on('inspect', inspect);
    return () => { bridge.off('inspect'); logger.close(); }
}
'''

In [ ]:
brd.logger.show(clear=True)

In [ ]:
brd.add_plugins(insp)

In [ ]:
insp.log("Look Ma', now greeny!")

In [ ]:
content, _ = insp.send(msg := insp.msg(ctx='inspect', kind='echo', data={'a': 1}), timeout=2)
test_eq(content, {'ctx': 'inspect', 'kind': 'echo', 'msg': msg, 'msg_id': msg['msg_id']})

We can re-load any plugin by calling `addPlugins` again. See above `insp` now uses a new logger.

In the examples folder, there's an [inspect](./examples/inspect_plugin.ipynb) plugin more fully developed.

# Loader
> Convenience python-side plugin for loading scripts and ESMs.

In [ ]:
#| exporti

class Loader(BridgePlugin):
    def __init__(self, 
            lnks: dict[str, FT]|None = None, 
            esms: dict[str, str|Path]|None = None
        ):
        self._lnks, self._esms, self._loaded = lnks or {}, esms or {}, {}
        super().__init__('loader')

    def on_init(self, *args, info:str, **kwargs):
        super().on_init(*args, info=info, **kwargs)
        if info == 'initialized':
            self.load_links(self._lnks)
            self.load(self._esms)
            self.log('Bridge initialized')

    @property
    def loading(self) -> bool: return len(tuple(self._pending.keys())) > 0
    @overload
    def loaded(self, name:str) -> bool: ...
    @overload
    def loaded(self, name:None=None) -> dict[str, str|None]: ...
    def loaded(self, name:str|None=None):
        if not name: return self._loaded.copy()
        return name in self._loaded
    
    def load(self, esms: dict[str, str|Path]|None=None, reload:bool=False, cache:bool=True):
        if not esms: return
        msg = self.msg(esms, ctx=self.ctx_name, cmd='load', 
            args={n:anysource(esm) for n,esm in esms.items()}, reload=reload, cache=cache)
        self.send(msg)
    def load_links(self, lnks: dict[str, FT]|None=None):
        if not lnks: return
        msg = self.msg(lnks, ctx=self.ctx_name, cmd='loadLinks', args={n:_to_js(_) for n,_ in lnks.items()})
        self.send(msg)

    async def _asend(self, msg, kind:str):
        res = await self.asend(msg)
        try:
            msg_id = res[0]['msg_id']  # type: ignore
            if msg_id in self._pending: self.log(f"Timeout loading {kind} {msg_id}")
        except Exception: pass
    async def aload(self, esms: dict[str, str|Path]|None=None, reload:bool=False, cache:bool=True):
        if not esms: return
        msg = self.msg(esms, ctx=self.ctx_name, cmd='load', 
            args={n:esm.read_text() if isinstance(esm, Path) else esm for n,esm in esms.items()}, 
                reload=reload, cache=cache)
        await self._asend(msg, 'ESMs')
    async def aload_links(self, lnks: dict[str, FT]|None=None):
        if not lnks: return
        msg = self.msg(lnks, ctx=self.ctx_name, cmd='loadLinks', args={n:_to_js(_) for n,_ in lnks.items()})
        await self._asend(msg, 'links')

    def on_load(self, *args, success:list[str], failed:list[dict[str,str]], tracker:dict, **kwargs):
        for name in success: self.log(f"'{name}' loaded.")
        for res in failed: self.log(f"'{res['name']}' failed: {res['error']}")
        self._loaded |= {n:None for n in success}  # don't store the source code for now
        
    def on_loadLinks(self, *args, success:list[str], failed:list[dict[str,str]], tracker:dict, **kwargs):
        for name in success: self.log(f"'{name}' loaded.")
        for res in failed: self.log(f"'{res['name']}' failed: {res['error']}")
        self._loaded |= {n:tracker[n] for n in success}
    
    # def on_loadPlugins(self, *args, success:list[str], failed:list[dict[str,str]], **kwargs):
    #     # for name in success: self.log(f"'{name}' loaded.")
    #     for res in failed: self.log(f"'{res['name']}' failed: {res['error']}")

In [ ]:
brd.logger.show(clear=True)

In [ ]:
loader = Loader(dict(
    test=Script('// debugger;\nconsole.log("test")', id='test-script'),
    test2=Script('// debugger;\nconsole.log(a)', id='test-script2')
))
brd.add_plugins(loader)
test_is(brd.loader.loading, True)
test_is(brd.loader, loader)

In [ ]:
loader.load_links({
    'test3': Script('// debugger;\nbridge.logger.log("test3")', id='test-script3'),
    'test4': Script('// debugger;\nbridge.logger.log(a)', id='test-script4')
})

In [ ]:
loader.load({'htmx1': '''
// debugger;
import htmx from "https://unpkg.com/htmx.org@next/dist/htmx.esm.js";
console.log(htmx);
''',
    'htmx2': '''
// debugger;
import htmx from "https://unpkg.com/htmx.org@next/dist/XXXX.esm.js";
console.log(htmx);
'''})

In [ ]:
blocks(lambda: loader.loaded('htmx1'), 2, show=_show)  # needed when running all above/below cells

test_eq(loader.loaded().keys(), set(('test', 'test3', 'htmx1')))

._.


In [ ]:
test_eq(brd.loader.loaded('htmx1'), True)
test_eq(brd.loader.loaded('nah'), False)

`Loader` can be used to execute any JS code (as an EcmaScript module). Note that unlike IPython `Javacript`, there's no output, the code won't run on opening the notebook until explicitly running the cell.

In [ ]:
brd.logger.show()

In [ ]:
display(HTML(
    '<button type="button" onclick="const canvas=document.getElementById(\'my-canvas\');\ncanvas.confetti({spread:70, particleCount:100, origin: { y: 1 }})">Fire!</button><br>'
    '<canvas id="my-canvas" width="1000px" height="200px"></canvas>'
))
time.sleep(0.5)
loader.load({'confetti': '''
import confetti from "https://esm.sh/canvas-confetti@1.6";

function randomInRange(min, max) {
  return Math.random() * (max - min) + min;
}

const canvas = document.getElementById('my-canvas');
canvas.confetti = canvas.confetti || confetti.create(canvas, { resize: true });
'''}, reload=True)

In [ ]:
loader.load({'beep': '''
// debugger;
export function beep() {
    var snd = new Audio("data:audio/wav;base64,//uQRAAAAWMSLwUIYAAsYkXgoQwAEaYLWfkWgAI0wWs/ItAAAGDgYtAgAyN+QWaAAihwMWm4G8QQRDiMcCBcH3Cc+CDv/7xA4Tvh9Rz/y8QADBwMWgQAZG/ILNAARQ4GLTcDeIIIhxGOBAuD7hOfBB3/94gcJ3w+o5/5eIAIAAAVwWgQAVQ2ORaIQwEMAJiDg95G4nQL7mQVWI6GwRcfsZAcsKkJvxgxEjzFUgfHoSQ9Qq7KNwqHwuB13MA4a1q/DmBrHgPcmjiGoh//EwC5nGPEmS4RcfkVKOhJf+WOgoxJclFz3kgn//dBA+ya1GhurNn8zb//9NNutNuhz31f////9vt///z+IdAEAAAK4LQIAKobHItEIYCGAExBwe8jcToF9zIKrEdDYIuP2MgOWFSE34wYiR5iqQPj0JIeoVdlG4VD4XA67mAcNa1fhzA1jwHuTRxDUQ//iYBczjHiTJcIuPyKlHQkv/LHQUYkuSi57yQT//uggfZNajQ3Vmz+Zt//+mm3Wm3Q576v////+32///5/EOgAAADVghQAAAAA//uQZAUAB1WI0PZugAAAAAoQwAAAEk3nRd2qAAAAACiDgAAAAAAABCqEEQRLCgwpBGMlJkIz8jKhGvj4k6jzRnqasNKIeoh5gI7BJaC1A1AoNBjJgbyApVS4IDlZgDU5WUAxEKDNmmALHzZp0Fkz1FMTmGFl1FMEyodIavcCAUHDWrKAIA4aa2oCgILEBupZgHvAhEBcZ6joQBxS76AgccrFlczBvKLC0QI2cBoCFvfTDAo7eoOQInqDPBtvrDEZBNYN5xwNwxQRfw8ZQ5wQVLvO8OYU+mHvFLlDh05Mdg7BT6YrRPpCBznMB2r//xKJjyyOh+cImr2/4doscwD6neZjuZR4AgAABYAAAABy1xcdQtxYBYYZdifkUDgzzXaXn98Z0oi9ILU5mBjFANmRwlVJ3/6jYDAmxaiDG3/6xjQQCCKkRb/6kg/wW+kSJ5//rLobkLSiKmqP/0ikJuDaSaSf/6JiLYLEYnW/+kXg1WRVJL/9EmQ1YZIsv/6Qzwy5qk7/+tEU0nkls3/zIUMPKNX/6yZLf+kFgAfgGyLFAUwY//uQZAUABcd5UiNPVXAAAApAAAAAE0VZQKw9ISAAACgAAAAAVQIygIElVrFkBS+Jhi+EAuu+lKAkYUEIsmEAEoMeDmCETMvfSHTGkF5RWH7kz/ESHWPAq/kcCRhqBtMdokPdM7vil7RG98A2sc7zO6ZvTdM7pmOUAZTnJW+NXxqmd41dqJ6mLTXxrPpnV8avaIf5SvL7pndPvPpndJR9Kuu8fePvuiuhorgWjp7Mf/PRjxcFCPDkW31srioCExivv9lcwKEaHsf/7ow2Fl1T/9RkXgEhYElAoCLFtMArxwivDJJ+bR1HTKJdlEoTELCIqgEwVGSQ+hIm0NbK8WXcTEI0UPoa2NbG4y2K00JEWbZavJXkYaqo9CRHS55FcZTjKEk3NKoCYUnSQ0rWxrZbFKbKIhOKPZe1cJKzZSaQrIyULHDZmV5K4xySsDRKWOruanGtjLJXFEmwaIbDLX0hIPBUQPVFVkQkDoUNfSoDgQGKPekoxeGzA4DUvnn4bxzcZrtJyipKfPNy5w+9lnXwgqsiyHNeSVpemw4bWb9psYeq//uQZBoABQt4yMVxYAIAAAkQoAAAHvYpL5m6AAgAACXDAAAAD59jblTirQe9upFsmZbpMudy7Lz1X1DYsxOOSWpfPqNX2WqktK0DMvuGwlbNj44TleLPQ+Gsfb+GOWOKJoIrWb3cIMeeON6lz2umTqMXV8Mj30yWPpjoSa9ujK8SyeJP5y5mOW1D6hvLepeveEAEDo0mgCRClOEgANv3B9a6fikgUSu/DmAMATrGx7nng5p5iimPNZsfQLYB2sDLIkzRKZOHGAaUyDcpFBSLG9MCQALgAIgQs2YunOszLSAyQYPVC2YdGGeHD2dTdJk1pAHGAWDjnkcLKFymS3RQZTInzySoBwMG0QueC3gMsCEYxUqlrcxK6k1LQQcsmyYeQPdC2YfuGPASCBkcVMQQqpVJshui1tkXQJQV0OXGAZMXSOEEBRirXbVRQW7ugq7IM7rPWSZyDlM3IuNEkxzCOJ0ny2ThNkyRai1b6ev//3dzNGzNb//4uAvHT5sURcZCFcuKLhOFs8mLAAEAt4UWAAIABAAAAAB4qbHo0tIjVkUU//uQZAwABfSFz3ZqQAAAAAngwAAAE1HjMp2qAAAAACZDgAAAD5UkTE1UgZEUExqYynN1qZvqIOREEFmBcJQkwdxiFtw0qEOkGYfRDifBui9MQg4QAHAqWtAWHoCxu1Yf4VfWLPIM2mHDFsbQEVGwyqQoQcwnfHeIkNt9YnkiaS1oizycqJrx4KOQjahZxWbcZgztj2c49nKmkId44S71j0c8eV9yDK6uPRzx5X18eDvjvQ6yKo9ZSS6l//8elePK/Lf//IInrOF/FvDoADYAGBMGb7FtErm5MXMlmPAJQVgWta7Zx2go+8xJ0UiCb8LHHdftWyLJE0QIAIsI+UbXu67dZMjmgDGCGl1H+vpF4NSDckSIkk7Vd+sxEhBQMRU8j/12UIRhzSaUdQ+rQU5kGeFxm+hb1oh6pWWmv3uvmReDl0UnvtapVaIzo1jZbf/pD6ElLqSX+rUmOQNpJFa/r+sa4e/pBlAABoAAAAA3CUgShLdGIxsY7AUABPRrgCABdDuQ5GC7DqPQCgbbJUAoRSUj+NIEig0YfyWUho1VBBBA//uQZB4ABZx5zfMakeAAAAmwAAAAF5F3P0w9GtAAACfAAAAAwLhMDmAYWMgVEG1U0FIGCBgXBXAtfMH10000EEEEEECUBYln03TTTdNBDZopopYvrTTdNa325mImNg3TTPV9q3pmY0xoO6bv3r00y+IDGid/9aaaZTGMuj9mpu9Mpio1dXrr5HERTZSmqU36A3CumzN/9Robv/Xx4v9ijkSRSNLQhAWumap82WRSBUqXStV/YcS+XVLnSS+WLDroqArFkMEsAS+eWmrUzrO0oEmE40RlMZ5+ODIkAyKAGUwZ3mVKmcamcJnMW26MRPgUw6j+LkhyHGVGYjSUUKNpuJUQoOIAyDvEyG8S5yfK6dhZc0Tx1KI/gviKL6qvvFs1+bWtaz58uUNnryq6kt5RzOCkPWlVqVX2a/EEBUdU1KrXLf40GoiiFXK///qpoiDXrOgqDR38JB0bw7SoL+ZB9o1RCkQjQ2CBYZKd/+VJxZRRZlqSkKiws0WFxUyCwsKiMy7hUVFhIaCrNQsKkTIsLivwKKigsj8XYlwt/WKi2N4d//uQRCSAAjURNIHpMZBGYiaQPSYyAAABLAAAAAAAACWAAAAApUF/Mg+0aohSIRobBAsMlO//Kk4soosy1JSFRYWaLC4qZBYWFRGZdwqKiwkNBVmoWFSJkWFxX4FFRQWR+LsS4W/rFRb/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////VEFHAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAU291bmRib3kuZGUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMjAwNGh0dHA6Ly93d3cuc291bmRib3kuZGUAAAAAAAAAACU=");  
    snd.play();
}
window.$beep = beep;
'''})
display(HTML('<button type="button" onclick="window.$beep()">Beep!</button>'))

# HTMX plugin

In [ ]:
#| export

class HTMXPlugin(BridgePlugin):
    ctx_name = 'htmx'
    sels = ['.output', '.jp-Cell-outputArea']
    url = 'https://cdn.jsdelivr.net/npm/htmx.org@2.0.3/dist/htmx.esm.js'

    def setup(self):
        self.send(self.msg(ctx=self.ctx_name, cmd='setup', args=[self.sels, self.url]))
    def on_info(self, *args, info:str, **kwargs):
        if info == 'setup': self.log(f"HTMX setup.")

In [ ]:
brd.logger.show(clear=True)

In [ ]:
brd.add_plugins(htmxp := HTMXPlugin())
blocks(lambda: htmxp.is_initialized is not None, 2, show=_show)  # needed when running all above/below cells

._.


True

In [ ]:
htmxp.setup()

# ObserverManager
> `MutationObserver` manager for `bridget`


In [ ]:
#| export

observer_js = BUNDLE_PATH / 'js/observer.js'

In [ ]:
bundled(observer_js)(debugger=DEBUG(), ts=True);

In [ ]:
brd.logger.show(clear=True)

In [ ]:
observer_scr = Script(bundled('''
import { getObserverManager } from './observer.js';
getObserverManager();
''')(), 
    type='module', id='brd-get-observer-manager')

In [ ]:
loader.load_links({'get_observer': observer_scr})  # load_links is async

ObserverManager is a normal ES module, just import it. But for local development in notebooks, or until we register it, we can use the `loader` or whatever of the many methods we now have to load JS code..


In [ ]:
loader.loaded()

{'test': script(('// debugger;\nconsole.log("test")',),{'id': 'test-script'}),
 'test3': script(('// debugger;\nbridge.logger.log("test3")',),{'id': 'test-script3'}),
 'htmx1': None,
 'confetti': None,
 'beep': None,
 'get_observer': script(('\nconst {getObserverManager} = await brdimport("./observer.js");\ngetObserverManager();\n',),{'type': 'module', 'id': 'brd-get-observer-manager'})}

In [ ]:
#| exporti

# @FC.patch
# async def brdimport(self: Loader, source: str|Path, name:str|None=None, base:str|Path|None=None):
#     src = resolve_ESM(source, base) if isinstance(source, str) else source
#     if src is None: raise ValueError(f"Invalid module specifier: {src}")
#     d = {}
#     if isinstance(src, ParseResult):
#         url = src.geturl()
#         d[name or source] = Script(type='module', src=url, id=name)
#         await self.aload_links(d)
#     else:
#         d[name or str(source)] = src
#         await self.aload(d)@FC.patch

@FC.patch
def brdimport(self: Loader, source: str|Path, name:str|None=None, base:str|Path|None=None):
    src = resolve_ESM(source, base) if isinstance(source, str) else source
    if src is None: raise ValueError(f"Invalid module specifier: {src}")
    d = {}
    if isinstance(src, ParseResult):
        url = src.geturl()
        d[name or source] = Script(type='module', src=url, id=name)
        self.load_links(d)
    else:
        d[name or str(source)] = src
        self.load(d)

In [ ]:
brd.logger.show()

In [ ]:
brd.loader.brdimport('./observer.js')
blocks(lambda: loader.loaded('./observer.js'), 2, show=_show)  # needed when running all above/below cells

._.


True

In [ ]:
test_eq(loader.loaded('./observer.js'), True)

We can also use with Python the equivalent to ES5 relative import declaration (don't forget to `bundled` your source or use `brdimport` directly).

**Front-end**:
```js
import { getObserverManager } from './observer.js';
const observer= getObserverManager();
```

**Kernel**:
```python
get_bridge().loader.brdimport('./observer.js')
```

In [ ]:
observer_plugin = anysource('''
export default async function initializeObserverPlugin(bridge) {
    const { getObserverManager } = await brdimport('./observer.js');
    return [null, { getObserverManager }]
}
''')

or:

In [ ]:
#| exporti

observer_plugin = bundled('''
import { getObserverManager } from './observer.js';
export default function initializeObserverPlugin(bridge) {
    return [null, { getObserverManager }]
}
''')()

In [ ]:
brd.logger.show(clear=True)

In [ ]:
brd.add_plugins(kwplugins={'observer':observer_plugin})

In [ ]:
[*brd.plugins.keys()]

['loader', 'htmx', 'fasthtmljs', 'badp', 'inspect', 'observer']

We could also add observerManager as a plugin.

# brd-mark
> Bridge plugin that defines a custom element that adds data- attributes to its parent and remove itself. It also processes the parent with htmx.

In [ ]:
brdmark_plugin = bundled(brdmark_js)()#(debugger=DEBUG(), ts=True)

We could load `brd_mark` directly, but as it depends on the bridge (for logging), better to load it as a plugin.

In [ ]:
brd.logger.show()

In [ ]:
brd.add_plugins(kwplugins={'brd_mark':brdmark_plugin})

In [ ]:
marker = 'aaaa<brd-mark id="marker-123">'
display(HTML(marker))

In [ ]:
print('asdf\nwerwqert')

asdf
werwqert


In [ ]:
display(HTML('<div class="bridge">aaaa</div><brd-mark id="marker-1234"></brd-mark>'))
display(HTML('<div class="bridge">bbbb</div><brd-mark id="marker-12345"></brd-mark>'))

In [ ]:
HTML(Brd_Mark(id=new_id()))

# get_bridge

In [ ]:
#| export

def show_summary(brd:Bridge):
    summ = shortens(map(to_xml, [*brd.plugins.keys(), *brd.loader.loaded().values()]), 'r', 140)
    display(HTML(ScriptsDetails(summ)))

In [ ]:
brd.logger.show()

In [ ]:
brd.loader.load_links({
    'surreal':bridge_scripts()['surreal'],
    'css_scope_inline':bridge_scripts()['css_scope_inline'],
    })

In [ ]:
#| export

bridge_default_plugins = [
    HTMXPlugin(),
    BridgePlugin('observer', observer_plugin), 
    BridgePlugin('brd_mark', bundled(brdmark_js)()),
]


@FC.delegates(Bridge.create, keep=True)  # type: ignore
def get_bridge(
    logger:NBLogger|None=None, 
    show_logger:bool=False,
    *,
    lnks: dict[str, FT]|None = None, 
    esms: dict[str, str|Path]|None = None,
    plugins:Sequence[BridgePlugin]|None=None, 
    kwplugins:dict[str, str]|None=None, 
    wait:int=0,  # seconds to wait for plugins/links/modules to load
    summary:bool=False,
    **kwargs,  # `Bridge.create` kwargs
):
    if not __brd__: 
        timeout, sleep = kwargs.pop('timeout', 3), kwargs.pop('sleep', 0.2)
        lnks = {**_bridge_scripts_extra(), **(lnks or {})}
        plugins = [Loader(), *bridge_default_plugins, *(plugins or ())]
        brd = Bridge.create(*plugins, kwplugins=kwplugins, logger=logger, show_logger=show_logger, 
            timeout=timeout, sleep=sleep, **kwargs)
    else:
        brd = __brd__
        if logger: brd.logger = logger
        if not logger and show_logger: brd.logger.show()
        brd.add_plugins(*(plugins or ()), kwplugins=kwplugins, wait=wait)
    brd.loader.load_links(lnks)
    brd.loader.load(esms)
    if wait: blocks(lambda: not brd.loader.loading, wait)
    if summary: show_summary(brd)
    return brd

In [ ]:
nbdev.show_doc(get_bridge)

---

[source](https://github.com/civvic/bridget/blob/main/bridget/bridge.py#L499){target="_blank" style="float:right; font-size:smaller"}

### get_bridge

>      get_bridge (logger:NBLogger|None=None, show_logger:bool=False,
>                  lnks:dict[str,FT]|None=None,
>                  esms:dict[str,str|Path]|None=None,
>                  plugins:Sequence[BridgePlugin]|None=None,
>                  kwplugins:dict[str,str]|None=None, wait:int=0,
>                  summary:bool=False, factory:Callable[...,Any]|None=None,
>                  timeout:float=10, sleep:float=0.2, n:int=10,
>                  show:Callable[[bool],None]|None=None, **kwargs)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| logger | NBLogger \| None | None |  |
| show_logger | bool | False |  |
| lnks | dict[str, FT] \| None | None |  |
| esms | dict[str, str \| Path] \| None | None |  |
| plugins | Sequence[BridgePlugin] \| None | None |  |
| kwplugins | dict[str, str] \| None | None |  |
| wait | int | 0 | seconds to wait for plugins/links/modules to load |
| summary | bool | False |  |
| factory | Callable[..., Any] \| None | None |  |
| timeout | float | 10 |  |
| sleep | float | 0.2 |  |
| n | int | 10 |  |
| show | Callable[[bool], None] \| None | None |  |
| kwargs | VAR_KEYWORD |  |  |

In [ ]:
test_is(get_bridge(show_logger=True), brd)

In [ ]:
#| export

if bridge_cfg.bootstrap: get_bridge(show_logger=True)

In [ ]:
brd.close()
test_is(__brd__, None)

In [ ]:
brd.logger.clear_log()

with bridge_cfg(bootstrap=True):
    brd = get_bridge(show_logger=True, wait=5, summary=True)

In [ ]:
html = '''
<div>💩 👻 No style.</div>

<div>
    <style> /* Simple example. */
        me { margin: 20px; }
        me div { font-size: 5rem; }
    </style>
    <div>👻</div>
</div>
'''

display(HTML(html))

Bridge automatically loads some JavaScript libraries:

1. HTMX
2. FastHTML core scripts
3. Awesome gnat's Scope and Surreal scripts


When importing `bridge`, if `bridge_cfg.bootstrap` is `True` or there is an environment variable `BRIDGET_BOOTSTRAP` set to a true-ish value, it will automatically create a bridge.

----
<!-- # Colophon -->

In [ ]:
#| hide
#| eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean
from pote.basic import bundle_path
from bridget.bridge_widget import bundled

In [ ]:
#| hide
#| eval: false

if FC.IN_NOTEBOOK:
    BUNDLE_PATH = bundle_path(__name__)
    for f in ['bcanvas', 'bridge', 'observer', 'brdmark']: bundled(BUNDLE_PATH / f'js/{f}.js')()
    nb_path = '14_bridge.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)